## Demo: Triggers in ParallelActivity
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment (simulation time needs to match the available weather data)
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes
In this example we won't use object classes

#### 3. Create objects
##### 3.1. Create site object(s)
No site objects are created

##### 3.2. Create vessel object(s)
No vessel objects are created

##### 3.3 Create activity/activities

In [3]:
# initialise registry
registry = {}

In [4]:
a = model.BasicActivity(
    env= my_env,
    name= "a",
    ID= "a",
    registry= registry,
    duration= 3,
)
a2 = model.BasicActivity(
    env= my_env,
    name= "a2",
    ID= "a2",
    registry= registry,
    duration= 2,
)
a3 = model.BasicActivity(
    env= my_env,
    name= "a3",
    ID= "a3",
    registry= registry,
    duration= 5,
)

c = model.BasicActivity(
    env= my_env,
    name= "c",
    ID= "c",
    registry= registry,
    duration= 1,
)

Pa=model.ParallelActivity(    
    env= my_env,
    name= "Pa",
    ID= "Pa",
    registry= registry,
    sub_processes= [a, a2, a3],
)

Sc=model.SequentialActivity(    
    env= my_env,
    name= "Sc",
    ID= "Sc",
    registry= registry,
    sub_processes= [c, Pa],
)

b = model.BasicActivity(
    ID= "b",
    env= my_env,
    name= "b",
    registry= registry,
    duration= 10,
    start_event=[
        {
            "name": "a",
            "type": "activity",
            "state": "done"
        }
    ]

)

Sb=model.SequentialActivity(    
    env= my_env,
    name= "Sb",
    ID= "Sb",
    registry= registry,
    sub_processes= [b],
)

#### 4. Register processes and run simpy

In [5]:
model.register_processes([Sc,Sb])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs

In [6]:
pd.DataFrame(Pa.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:01,Pa,START,{},{}
1,1970-01-01 00:00:01,Pa,START,{},"{'type': 'subprocess', 'ref': 'a'}"
2,1970-01-01 00:00:01,Pa,START,{},"{'type': 'subprocess', 'ref': 'a2'}"
3,1970-01-01 00:00:01,Pa,START,{},"{'type': 'subprocess', 'ref': 'a3'}"
4,1970-01-01 00:00:03,Pa,STOP,{},"{'type': 'subprocess', 'ref': 'a2'}"
5,1970-01-01 00:00:04,Pa,STOP,{},"{'type': 'subprocess', 'ref': 'a'}"
6,1970-01-01 00:00:06,Pa,STOP,{},"{'type': 'subprocess', 'ref': 'a3'}"
7,1970-01-01 00:00:06,Pa,STOP,{},{}


In [7]:
pd.DataFrame(Sc.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,Sc,START,{},{}
1,1970-01-01 00:00:00,Sc,START,{},"{'type': 'subprocess', 'ref': 'c'}"
2,1970-01-01 00:00:01,Sc,STOP,{},"{'type': 'subprocess', 'ref': 'c'}"
3,1970-01-01 00:00:01,Sc,START,{},"{'type': 'subprocess', 'ref': 'Pa'}"
4,1970-01-01 00:00:06,Sc,STOP,{},"{'type': 'subprocess', 'ref': 'Pa'}"
5,1970-01-01 00:00:06,Sc,STOP,{},{}


In [8]:
pd.DataFrame(c.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,c,START,{},{}
1,1970-01-01 00:00:01,c,STOP,{},{}


In [9]:
pd.DataFrame(Sb.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,Sb,START,{},{}
1,1970-01-01 00:00:00,Sb,START,{},"{'type': 'subprocess', 'ref': 'b'}"
2,1970-01-01 00:00:14,Sb,STOP,{},"{'type': 'subprocess', 'ref': 'b'}"
3,1970-01-01 00:00:14,Sb,STOP,{},{}


In [10]:
pd.DataFrame(b.log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,b,WAIT_START,{},{}
1,1970-01-01 00:00:04,b,WAIT_STOP,{},{}
2,1970-01-01 00:00:04,b,START,{},{}
3,1970-01-01 00:00:14,b,STOP,{},{}


##### 5.2 Visualise gantt charts

In [11]:
plot.get_gantt_chart([Sc,c,Pa,a,a2,a3,b])